### Largest Empty Circle

In [1]:
%matplotlib ipympl

import numpy as np
from scipy import spatial
from shapely.geometry import asPolygon, asPoint
from matplotlib import path

import matplotlib.pyplot as plt

### Develop Function

In [2]:
# load test dat
points = np.loadtxt("test_dat.csv", delimiter=",")

In [3]:
%%time
# Step 1: get Voronoi vertices (scipy.spatial)
vor_Verts = spatial.Voronoi(points).vertices

CPU times: user 3.21 ms, sys: 0 ns, total: 3.21 ms
Wall time: 2.14 ms


In [7]:
%%time
# Step 2: select Voronoi vertices inside the convex hull (shapely)   
vor_shp = [asPoint(xy) for xy in vor_Verts]
hull_shp = asPolygon(points)
selector = [xy.within(hull_shp) for xy in vor_shp]
vor_Verts = vor_Verts[selector]  # Verts inside Polygon

CPU times: user 1.18 s, sys: 291 µs, total: 1.18 s
Wall time: 1.18 s


In [8]:
%%time
# Step 2: select Voronoi vertices inside the convex hull (mpl)
polygon = path.Path(points)
selector = polygon.contains_points(vor_Verts)
vor_Verts = vor_Verts[selector]  # Verts inside Polygon

CPU times: user 0 ns, sys: 1.18 ms, total: 1.18 ms
Wall time: 1.23 ms


In [9]:
%%time
# Step 3: Get nearest neighbors as well as radius und centroid of LEC
tree = spatial.KDTree(points)
neighbores = tree.query(vor_Verts)
idx = np.argmax(neighbores[0])
LEC_Radius = neighbores[0][idx]
LEC_Centroid = vor_Verts[idx]

CPU times: user 42.5 ms, sys: 0 ns, total: 42.5 ms
Wall time: 42.4 ms


In [10]:
fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.plot(points[:, 0], points[:, 1], 'go-', lw=1, ms=2)

plt.plot(LEC_Centroid[0], LEC_Centroid[1], 'bx')
circle = plt.Circle(LEC_Centroid, LEC_Radius, ec='blue', color='None' ,lw=1)
ax.add_artist(circle);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def pyLEC(points):
    '''
    LEC: Largest Empty Circle between Points in a flat plane
    also know as Toxic Waste Dump or Pole of Inaccessibility
    problem
    

    see: https://en.wikipedia.org/wiki/Largest_empty_sphere
    The largest empty circle problem is the problem of finding
    a circle of largest radius in the plane whose interior
    does not overlap with any given point and whose center
    is lying in the convex hull of the points.
    
    Hint: The LEC is centered at the internal Voronoi vertices
    with the largest shortest distance to the polygon

    Parameters
    ----------
    points : 2d numpy array with x, y coordinate of the points
             on the Concave Hull

    Returns
    -------
    LEC_Radius: scalar
    LEC_Centroid: (x, y)
    '''
    
    # Error handling
    if points.shape[0] < 3:
        print('less than 3 Points: ')
        return np.nan, (np.nan, np.nan)

    # Step 1: get Voronoi vertices (scipy.spatial)
    vor_Verts = spatial.Voronoi(points).vertices

    # Step 2: select Voronoi vertices inside the convex hull (mpl)
    polygon = path.Path(points)
    selector = polygon.contains_points(vor_Verts)
    vor_Verts = vor_Verts[selector]  # Verts inside Polygon Verts inside Polygon

    # Step 3: Get nearest neighbors as well as radius und centroid of LEC
    tree = spatial.KDTree(points)
    neighbores = tree.query(vor_Verts)
    idx = np.argmax(neighbores[0])
    LEC_Radius = neighbores[0][idx]
    LEC_Centroid = vor_Verts[idx]
    
    return LEC_Radius, LEC_Centroid

### test function

In [13]:
xy = np.loadtxt("test_dat.csv", delimiter=",")

LEC = pyLEC(xy)

fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.plot(xy[:, 0], xy[:, 1], 'go-', lw=1, ms=2)

plt.plot(LEC[1][0], LEC[1][1], 'bx')
circle = plt.Circle(LEC[1], LEC[0], ec='blue', color='None' ,lw=1)
ax.add_artist(circle);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …